# NEW TRAINER

In [1]:
from sage.config import load_config
from sage.training.trainer import MRITrainer

cfg = load_config()
cfg.augment = True
cfg.use_amp = False
cfg.phase_config = {
    'epochs': [200],
    'update': [['reg']]
}
cfg.encoder = {
    'name': 'convit',
    'config': {
        'embed_dim': 64,
        'local_up_to_layer': 4,
        'depth': 5,
        'num_heads': 8
    }
}
cfg.reg_opt.lr = 1e-4
trainer = MRITrainer(cfg)

Use cuda:0 as a device.
Output from encoder is 64.
Total Number of parameters: 530742
TOTAL TRAIN 2322 | VALID 291
MIXED PRECISION:: False


In [2]:
import wandb
wandb.login()
wandb.init(project='3d_smri',
           config=vars(cfg),
           name='ConViT_64_aug_extension',
#            tags=[]
  )

wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


In [ ]:
trainer.run(cfg)

## from checkpoint

In [3]:
PREFIX = 'G:/My Drive/brain_data/workspace/result/models/20210817-1239/'
SUFFIX = 'ep139_mae7.95.pt'

checkpoint = {
    'resume_epoch': 0,
    'models':{
        'encoder': f'{PREFIX}/encoder/{SUFFIX}',
        'domainer': f'{PREFIX}/domainer/{SUFFIX}',
        'regressor': f'{PREFIX}/regressor/{SUFFIX}',
    }
}

# FIND THIS IN WANDB.AI HOMEPAGE

resume_id = None

In [ ]:
import wandb
wandb.login()
wandb.init(project='3d_smri',
           config=vars(cfg),
           resume=resume_id,
  )

In [4]:
trainer.run(checkpoint=checkpoint)

Encoder is successfully loaded
Domainer is successfully loaded
Regressor is successfully loaded
Epoch 1 / 200 (phase: 0) BEST MAE inf
[train] 353.9 sec [valid] 7.4 sec 
Epoch 2 / 200 (phase: 0) BEST MAE 8.090
[train] 366.9 sec [valid] 7.6 sec 
Epoch 3 / 200 (phase: 0) BEST MAE 7.997
[train] 371.9 sec [valid] 8.3 sec 
Epoch 4 / 200 (phase: 0) BEST MAE 7.997
[train] 347.9 sec [valid] 8.3 sec 
Epoch 5 / 200 (phase: 0) BEST MAE 7.997
[train] 358.2 sec [valid] 7.5 sec 
Saved Models
Epoch 6 / 200 (phase: 0) BEST MAE 7.997
[train] 356.4 sec [valid] 8.0 sec 
Epoch 7 / 200 (phase: 0) BEST MAE 7.797
[train] 344.3 sec [valid] 7.8 sec 
Epoch 8 / 200 (phase: 0) BEST MAE 7.797
[train] 343.8 sec [valid] 7.6 sec 
Epoch 9 / 200 (phase: 0) BEST MAE 7.797
[train] 354.3 sec [valid] 7.4 sec 
Epoch 10 / 200 (phase: 0) BEST MAE 7.797
[train] 363.0 sec [valid] 7.4 sec 
Saved Models
Epoch 11 / 200 (phase: 0) BEST MAE 7.797
[train] 353.5 sec [valid] 7.5 sec 
Epoch 12 / 200 (phase: 0) BEST MAE 7.797
[train] 358.

train_rmse,7.2949
train_mae,5.1675
train_corr,0.9169
train_r2,0.8407
train_auc,0.57199
train_acc,0.37812
valid_rmse,9.96006
valid_mae,7.72417
valid_corr,0.83751
valid_r2,0.69416
valid_auc,0.61104


train_rmse,▇▇▅▆▆█▆▇▄▅▆▆▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▃▃▂▁▁▄▂▁▁▃▂▁▂
train_mae,▇▇▆▆▇█▆▇▅▅▆▆▅▅▅▄▄▄▄▄▄▃▄▃▃▃▂▃▃▂▁▁▃▂▁▂▃▂▁▁
train_corr,▃▃▄▃▃▁▃▂▆▄▄▄▄▅▅▅▅▆▅▆▅▆▆▆▆▆▇▆▆█▇█▆▇██▇▇█▇
train_r2,▃▃▄▃▃▁▃▂▅▄▄▄▄▅▅▅▅▅▅▆▅▆▆▆▆▆▇▆▆█▇█▆▇██▇▇█▇
train_auc,█▇▇█▇▇▇▆▆▅▆▅▄▆▅▄▄▅▄▄▅▄▃▄▃▁▂▂▃▂▂▃▁▃▃▂▂▃▂▁
train_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_rmse,▃▃▅▇▄▅▄▅▅▄▄▃▄▅▅▄▆▃▅▄▃▃▆▃▂▁▁▆▂▄▄▃▃▄▁▄▇█▄▃
valid_mae,▄▃▅▆▅▅▄▅▄▄▄▃▄▆▄▄▅▃▅▅▃▂▆▃▂▁▁▆▂▅▄▃▃▄▂▅██▃▂
valid_corr,▆▆▅▅▅▅▄▃▃▆▃▇▃▆▅▄▂▆▁▅▄▇▃▅▇█▇▅▆▃▃▅▅▅▇▆▅▂▄▅
valid_r2,▆▇▅▃▅▅▅▅▅▅▅▆▅▄▅▅▃▆▄▅▆▆▃▆▇█▇▄▇▅▅▆▆▅▇▅▂▁▅▆
valid_auc,▇▇▆▆▇█▆▅▅▆▅▆▄▇▄▅▅▆▃▆▅▄▄▃▃▄▄▄▃▃▃▄▂▄▄▂▄▁▃▅


## Grid Search

In [1]:
import wandb

from sage.config import load_config
from sage.training.trainer import MRITrainer

sweep_config = {
    "name" : "ConViT",
    "method" : "grid",
    "metric": {
        "name": "valid_mae",
        "goal": "minimize"
    },
    "parameters" : {
        "embed_dim" :{
            "values": [48, 64]
        },
        "depth": {
            "values": [2, 4]
        },
        "learning_rate": {
            "values": [1e-2, 1e-3, 1e-4]
        }
      }
    }

sweep_id = wandb.sweep(sweep_config, project='3d_smri')

Create sweep with ID: cmjnmxvp
Sweep URL: https://wandb.ai/1pha/3d_smri/sweeps/cmjnmxvp


In [2]:
from IPython.display import clear_output

def run_sweep():
    
    with wandb.init():

        cfg = load_config()
        
        # PRESET
        cfg.encoder = {
            'name': 'convit',
            'config': {
                'num_heads': 8
            }
        }
        cfg.phase_config = {
            'epochs': [150],
            'update': [['reg']]
        }
        
        # LOAD COMBINATIONS
        convit_cfg = wandb.config
        __cfg = dict()
        __cfg.update(convit_cfg)
        __cfg['local_up_to_layer'] = __cfg['depth'] - 1
        
        cfg.reg_opt.lr = __cfg.pop('learning_rate')
        cfg.encoder.config.update(__cfg)
        
        # DEFINE TRAINER
        trainer = MRITrainer(cfg)
                       
        # WANDB SETUP
        name = f'ConViT-embed_dim{cfg.encoder.config.embed_dim}_depth{cfg.encoder.config.depth}'
        wandb.run.name = name
        wandb.config.update(cfg)
        
        # RUN
        trainer.run(cfg)
        clear_output()

In [3]:
wandb.agent(sweep_id, function=run_sweep)

wandb: Agent Starting Run: 6lrbrnt9 with config:
wandb: 	depth: 2
wandb: 	embed_dim: 48
wandb: 	learning_rate: 0.01
wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


Use cuda:0 as a device.


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


Output from encoder is 48.
Total Number of parameters: 266538
TOTAL TRAIN 1161 | VALID 291
MIXED PRECISION:: True
Epoch 1 / 150 (phase: 0) BEST MAE inf
[train] 57.2 sec [valid] 7.1 sec 
Epoch 2 / 150 (phase: 0) BEST MAE 16.454
[train] 49.4 sec [valid] 7.9 sec 
Epoch 3 / 150 (phase: 0) BEST MAE 16.217
[train] 51.6 sec [valid] 7.4 sec 
Epoch 4 / 150 (phase: 0) BEST MAE 16.199
[train] 49.6 sec [valid] 7.6 sec 
Epoch 5 / 150 (phase: 0) BEST MAE 16.188
[train] 52.6 sec [valid] 7.1 sec 
Epoch 6 / 150 (phase: 0) BEST MAE 16.181
[train] 49.6 sec [valid] 7.3 sec 
Epoch 7 / 150 (phase: 0) BEST MAE 16.172
[train] 52.2 sec [valid] 7.1 sec 
Epoch 8 / 150 (phase: 0) BEST MAE 16.172
[train] 48.9 sec [valid] 7.1 sec 
Epoch 9 / 150 (phase: 0) BEST MAE 16.167
[train] 52.8 sec [valid] 7.1 sec 
Epoch 10 / 150 (phase: 0) BEST MAE 16.166
[train] 49.1 sec [valid] 7.1 sec 
Epoch 11 / 150 (phase: 0) BEST MAE 16.164
[train] 52.6 sec [valid] 7.1 sec 
Epoch 12 / 150 (phase: 0) BEST MAE 16.164
[train] 49.1 sec [va

train_rmse,18.65057
train_mae,15.42729
train_corr,0.01916
train_r2,-0.01909
train_auc,0.55242
train_acc,0.37812
valid_rmse,18.95937
valid_mae,16.16649
valid_corr,0.28549
valid_r2,0.00113
valid_auc,0.7042


...

wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


[valid] 6.9 sec 
Waited for 3 times no better result 3 / 3 at EPOCH 143


train_rmse,6.72583
train_mae,5.078
train_corr,0.93426
train_r2,0.8728
train_auc,0.67991
train_acc,0.34453
valid_rmse,13.14051
valid_mae,10.67568
valid_corr,0.72222
valid_r2,0.49979
valid_auc,0.67505


train_rmse,██▆▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train_mae,██▆▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_corr,▁▁▁▁▁▁▁▁▁▁▁▂▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███████████
train_r2,▁▁▄▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████████
train_auc,▂▃▁▁▂▂▁▁▁▂▄▅▆▆▆▆▆▆▇█▇███████▇▇▆▆▆▆▆▆▆▆▆▆
train_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▄█▇▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅
valid_rmse,██▆▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_mae,██▅▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_corr,▃▅▅▅▁▁▁▁▁▁▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██████████████
valid_r2,▁▂▄▇████████████████████████████████████
valid_auc,▆▆▁▁▃▃▃▃▃▃▇▇▇▆▆▆▆▆▇▇███████▇▇▇▆▆▆▆▆▆▆▆▆▆


Run 9fz2q5dg errored: FileNotFoundError(2, 'No such file or directory')
wandb: ERROR Run 9fz2q5dg errored: FileNotFoundError(2, 'No such file or directory')
Detected 5 failed runs in a row at start, killing sweep.
wandb: ERROR Detected 5 failed runs in a row at start, killing sweep.
wandb: To change this value set WANDB_AGENT_MAX_INITIAL_FAILURES=val


# NAIVE LEARNING

In [1]:
import os
import wandb

from sage.config import *
from sage.training.runner import run
from utils.misc import seed_everything, get_today
from IPython.display import clear_output

In [2]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = []
cfg.plot = False

cfg.augment = True
cfg.epochs = 200
cfg.model_name = 'resnet_no_maxpool'
cfg.early_patience = 20
cfg.start_channels = 32
cfg.weight_decay = 0.01

In [3]:
# checkpoint = {
#     'resume_epoch': 45,
#     'path': '../result/models/savetest20210608-1451/resnet_no_maxpool_ep45_mae5.86.pth'
# }

In [3]:
name = 'RES(256) DALLAS ADD'
cfg.RESULT_PATH = os.path.join(cfg.RESULT_PATH, name + get_today())

In [4]:
wandb.login()
wandb.init(project='3d_smri',
           config=vars(cfg),
           name=name
  )

wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


In [6]:
run(cfg)

Model Resnet_no_maxpool is selected.
TOTAL TRAIN 2322 | VALID 291
Epoch 1 / 200, BEST MAE inf
[train] 285.7 sec [valid] 13.7 sec 
TRAIN :: LOSS 3230.369 | RMSE 56.81 | MAE 53.91 | R2 -8.37 | CORR 0.41
VALID :: LOSS 3033.977 | RMSE 55.12 | MAE 52.13 | R2 -7.37 | CORR 0.66
Saving ...
Epoch 2 / 200, BEST MAE 52.133
[train] 242.4 sec [valid] 10.3 sec 
TRAIN :: LOSS 2926.768 | RMSE 54.07 | MAE 51.21 | R2 -7.49 | CORR 0.57
VALID :: LOSS 2962.794 | RMSE 54.48 | MAE 51.51 | R2 -7.18 | CORR 0.71
Saving ...
Epoch 3 / 200, BEST MAE 51.509
[train] 257.0 sec [valid] 9.0 sec 
TRAIN :: LOSS 2647.198 | RMSE 51.42 | MAE 48.65 | R2 -6.68 | CORR 0.68
VALID :: LOSS 2517.612 | RMSE 50.23 | MAE 47.31 | R2 -5.95 | CORR 0.83
Saving ...
Epoch 4 / 200, BEST MAE 47.312
[train] 280.2 sec [valid] 9.5 sec 
TRAIN :: LOSS 2355.147 | RMSE 48.50 | MAE 45.89 | R2 -5.83 | CORR 0.75
VALID :: LOSS 2163.654 | RMSE 46.57 | MAE 43.93 | R2 -4.97 | CORR 0.88
Saving ...
Epoch 5 / 200, BEST MAE 43.927
[train] 252.3 sec [valid] 9.

[train] 239.7 sec [valid] 8.6 sec 
TRAIN :: LOSS 36.543 | RMSE 6.04 | MAE 4.68 | R2 0.89 | CORR 0.95
VALID :: LOSS 87.010 | RMSE 9.46 | MAE 7.49 | R2 0.75 | CORR 0.93
Saving ...
Epoch 40 / 200, BEST MAE 5.493
[train] 236.3 sec [valid] 9.0 sec 
TRAIN :: LOSS 35.175 | RMSE 5.91 | MAE 4.58 | R2 0.90 | CORR 0.95
VALID :: LOSS 61.803 | RMSE 7.99 | MAE 6.19 | R2 0.82 | CORR 0.92
Saving ...
Epoch 41 / 200, BEST MAE 5.493
[train] 246.6 sec [valid] 8.8 sec 
TRAIN :: LOSS 37.368 | RMSE 6.10 | MAE 4.67 | R2 0.89 | CORR 0.94
VALID :: LOSS 58.893 | RMSE 7.71 | MAE 6.04 | R2 0.84 | CORR 0.92
Saving ...
Epoch 42 / 200, BEST MAE 5.493
[train] 242.7 sec [valid] 8.8 sec 
TRAIN :: LOSS 36.505 | RMSE 6.03 | MAE 4.63 | R2 0.89 | CORR 0.95
VALID :: LOSS 52.861 | RMSE 7.32 | MAE 5.72 | R2 0.85 | CORR 0.93
Saving ...
Epoch 43 / 200, BEST MAE 5.493
[train] 231.7 sec [valid] 8.7 sec 
TRAIN :: LOSS 34.139 | RMSE 5.84 | MAE 4.47 | R2 0.90 | CORR 0.95
VALID :: LOSS 63.813 | RMSE 8.13 | MAE 6.25 | R2 0.82 | CORR 0.

train_rmse,5.13191
train_mae,3.88871
train_r2,0.92349
train_corr,0.96108
train_loss,26.28916
valid_loss,51.09708
valid_rmse,7.0481
valid_mae,5.6217
valid_r2,0.86312
valid_corr,0.93105
_runtime,14467


train_rmse,██▇▆▆▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mae,██▇▆▆▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_r2,▁▂▂▄▄▅▆▇▇▇██████████████████████████████
train_corr,▁▃▄▆▆▇▇▇▇▇▇▇▇▇██████████████████████████
train_loss,█▇▇▅▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,██▇▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_rmse,██▇▇▅▅▄▃▄▃▃▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_mae,██▇▇▅▅▄▃▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_r2,▁▁▂▃▅▆▆▇▇▇▇█████████████████████████████
valid_corr,▁▂▅▇▇▇▇▇▆▇███▇██████████████████████████
_runtime,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


ResNet(
  (conv1): Conv3d(1, 32, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=Tru

# EFFICIENTNET

In [1]:
import os
import wandb

from sage.config import *
from sage.training.runner import run
from utils.misc import seed_everything, get_today
from IPython.display import clear_output

In [2]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = []
cfg.plot = False

cfg.augment = True
cfg.epochs = 200
cfg.model_name = 'efficientnet-b0'
cfg.early_patience = 30

In [3]:
checkpoint = {
    'resume_epoch': 78,
    'path': '../result/models/EFFICIENTNET TEST20210613-2121/efficientnet-b0_ep77_mae7.69.pth'
}

In [4]:
name = 'EFFICIENTNET TEST RESUME 78'
cfg.RESULT_PATH = os.path.join(cfg.RESULT_PATH, name + get_today())

In [5]:
wandb.login()
wandb.init(project='3d_smri',
           config=vars(cfg),
           name=name
  )

wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [6]:
run(cfg, checkpoint)

Model Efficientnet-b0 is selected.
TOTAL TRAIN 2322 | VALID 291
Epoch 79 / 200, BEST MAE inf
[train] 297.6 sec [valid] 11.9 sec 
TRAIN :: LOSS 48.788 | RMSE 7.00 | MAE 5.14 | R2 0.86 | CORR 0.93
VALID :: LOSS 113.676 | RMSE 10.66 | MAE 8.78 | R2 0.69 | CORR 0.90
Saving ...
Epoch 80 / 200, BEST MAE 8.777
[train] 288.4 sec [valid] 12.2 sec 
TRAIN :: LOSS 49.436 | RMSE 7.05 | MAE 5.21 | R2 0.86 | CORR 0.93
VALID :: LOSS 124.287 | RMSE 11.14 | MAE 9.09 | R2 0.66 | CORR 0.89
Saving ...
Epoch 81 / 200, BEST MAE 8.777
[train] 296.3 sec [valid] 12.5 sec 
TRAIN :: LOSS 50.444 | RMSE 7.12 | MAE 5.17 | R2 0.85 | CORR 0.92
VALID :: LOSS 104.505 | RMSE 10.22 | MAE 8.28 | R2 0.71 | CORR 0.90
Saving ...
Epoch 82 / 200, BEST MAE 8.282
[train] 280.1 sec [valid] 11.4 sec 
TRAIN :: LOSS 46.338 | RMSE 6.82 | MAE 5.07 | R2 0.86 | CORR 0.93
VALID :: LOSS 96.752 | RMSE 9.94 | MAE 8.02 | R2 0.73 | CORR 0.87
Saving ...
Epoch 83 / 200, BEST MAE 8.021
[train] 287.2 sec [valid] 12.0 sec 
TRAIN :: LOSS 47.388 | RM

[train] 287.1 sec [valid] 11.4 sec 
TRAIN :: LOSS 30.440 | RMSE 5.53 | MAE 4.05 | R2 0.91 | CORR 0.95
VALID :: LOSS 99.226 | RMSE 9.94 | MAE 8.11 | R2 0.73 | CORR 0.90
Saving ...
Epoch 119 / 200, BEST MAE 7.410
[train] 293.7 sec [valid] 11.5 sec 
TRAIN :: LOSS 30.661 | RMSE 5.55 | MAE 4.05 | R2 0.91 | CORR 0.95
VALID :: LOSS 84.560 | RMSE 9.17 | MAE 7.47 | R2 0.77 | CORR 0.90
Saving ...
Epoch 120 / 200, BEST MAE 7.410
[train] 305.8 sec [valid] 11.6 sec 
TRAIN :: LOSS 30.559 | RMSE 5.54 | MAE 4.06 | R2 0.91 | CORR 0.95
VALID :: LOSS 94.010 | RMSE 9.69 | MAE 7.95 | R2 0.74 | CORR 0.90
Saving ...
Epoch 121 / 200, BEST MAE 7.410
[train] 286.3 sec [valid] 11.8 sec 
TRAIN :: LOSS 29.863 | RMSE 5.48 | MAE 4.07 | R2 0.91 | CORR 0.96
VALID :: LOSS 104.936 | RMSE 10.07 | MAE 8.24 | R2 0.72 | CORR 0.90
Saving ...
Epoch 122 / 200, BEST MAE 7.410
[train] 282.7 sec [valid] 11.3 sec 
TRAIN :: LOSS 26.777 | RMSE 5.19 | MAE 3.79 | R2 0.92 | CORR 0.96
VALID :: LOSS 105.450 | RMSE 10.12 | MAE 8.24 | R2 0

train_rmse,5.10696
train_mae,3.72215
train_r2,0.92423
train_corr,0.96142
train_loss,25.92911
valid_loss,99.7807
valid_rmse,9.9606
valid_mae,8.27774
valid_r2,0.72663
valid_corr,0.89647
_runtime,16661


train_rmse,███▇▇▇▆▇▅▆▇▅▅▆▅▄▅▄▅▄▃▃▃▄▃▃▃▃▃▃▂▃▂▂▂▂▁▁▁▁
train_mae,████▇▇▆▆▅▆▇▅▅▅▅▅▅▄▅▄▃▃▃▄▃▃▃▃▃▃▂▃▂▂▂▂▁▂▁▁
train_r2,▁▁▁▂▃▂▄▃▅▄▃▄▅▄▄▅▅▅▅▆▆▆▆▅▆▆▆▆▆▆█▆▇▇▇▇████
train_corr,▁▁▁▂▃▂▄▃▅▄▃▄▅▄▄▅▅▅▅▆▆▆▆▅▆▆▆▆▆▆█▆▇▇▇▇████
train_loss,███▇▆▇▅▆▄▅▆▅▄▅▅▄▄▄▄▃▃▃▃▄▃▃▃▃▃▃▁▃▂▂▂▂▁▁▁▁
valid_loss,▆█▅▆▆▄▃▅▅▃▄▃▄▂▄█▄▇▁▄▃▄▂▆▂▄▂▅▂▃▅▄▅▃▆▄▃▄▃▄
valid_rmse,▆█▅▇▆▄▃▅▅▃▄▃▅▂▄█▄▇▁▄▃▄▂▆▂▄▂▅▂▃▅▄▅▄▆▄▃▄▂▄
valid_mae,▆▇▄▆▆▄▃▅▄▃▃▂▄▂▄█▃▆▁▃▂▄▂▅▁▃▁▄▁▃▄▄▄▃▆▃▃▃▂▄
valid_r2,▃▁▄▂▃▅▆▄▅▆▅▆▄▇▅▁▅▂█▅▆▅▇▃▇▅▇▅▇▆▅▅▄▆▃▆▆▆▇▅
valid_corr,▅▄▅▂▄▂▆▄▄▂▅▅▂▅▆▄▄▅▄▆▃▄▄▆▅▄▇▅▅▆▆▅█▄▁▅▇▆▅▅
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


EfficientNet3D(
  (_conv_stem): Conv3dStaticSamePadding(
    1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock3D(
      (_depthwise_conv): Conv3dStaticSamePadding(
        32, 32, kernel_size=(3, 3, 3), stride=[2, 2, 2], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(0, 1, 0, 1, 0, 1), value=0.0)
      )
      (_bn1): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv3dStaticSamePadding(
        32, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv3dStaticSamePadding(
        8, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv3dStaticS

In [7]:
wandb.finish()

## SWEEP

In [2]:
import wandb

sweep_config = {
    "name" : "Resnet",
    "method" : "grid", #grid, random
    "metric": { #목표로 삼을 매트릭에 대한 정보를 입력합니다.
        "name": "valid_mae",
        "goal": "minimize"
    },
    "parameters" : { #실험해볼 하이퍼 파라미터의 조합입니다.
        "optimizer" : {
            "values" : ['adam', 'adamW']
        },
        "weight_decay" :{
            "values": [0, 0.01]
        },
        "start_channels" :{
            "values": [8, 16, 32]
        }
      }
    }

sweep_id = wandb.sweep(sweep_config, project='3d_smri')

Create sweep with ID: i7iv565s
Sweep URL: https://wandb.ai/1pha/3d_smri/sweeps/i7iv565s


In [3]:
def run_sweep():
    
    with wandb.init():

        cfg = load_config()
        seed_everything(cfg.seed)
        cfg.registration = 'mni'
        cfg.unused_src = []
        cfg.plot = False

        cfg.augment = True
        cfg.epochs = 100
        cfg.model_name = 'resnet_no_maxpool'
        cfg.early_patience = 20
        
        _cfg = wandb.config
        __cfg = dict()
        __cfg.update(_cfg)
        cfg.update(__cfg)
        
        name = f'SWEEP_OPT-{cfg.optimizer}_WD{cfg.weight_decay}_SC{cfg.start_channels}'
        cfg.RESULT_PATH = os.path.join(cfg.RESULT_PATH, name + get_today())
        
        wandb.run.name = name
        wandb.config.update(cfg)
        
        run(cfg)
        
        clear_output()

In [4]:
wandb.agent(sweep_id, function=run_sweep)

wandb: Agent Starting Run: mwm3fxzu with config:
wandb: 	optimizer: adamW
wandb: 	start_channels: 32
wandb: 	weight_decay: 0.01


Model Resnet_no_maxpool is selected.
TOTAL TRAIN 2282 | VALID 286
Epoch 1 / 100, BEST MAE inf
[train] 241.1 sec [valid] 10.0 sec 
TRAIN :: LOSS 3205.938 | RMSE 56.62 | MAE 53.71 | R2 -8.21 | CORR 0.43
VALID :: LOSS 2867.374 | RMSE 53.57 | MAE 50.49 | R2 -6.73 | CORR 0.68
Saving ...
Epoch 2 / 100, BEST MAE 50.491
[train] 248.5 sec [valid] 9.9 sec 
TRAIN :: LOSS 2902.232 | RMSE 53.87 | MAE 51.00 | R2 -7.34 | CORR 0.59
VALID :: LOSS 2603.441 | RMSE 51.04 | MAE 47.90 | R2 -6.02 | CORR 0.72
Saving ...
Epoch 3 / 100, BEST MAE 47.897
[train] 243.5 sec [valid] 10.0 sec 
TRAIN :: LOSS 2631.244 | RMSE 51.30 | MAE 48.50 | R2 -6.56 | CORR 0.69
VALID :: LOSS 2548.353 | RMSE 50.50 | MAE 47.73 | R2 -5.87 | CORR 0.79
Saving ...
Epoch 4 / 100, BEST MAE 47.728
[train] 231.5 sec [valid] 9.8 sec 
TRAIN :: LOSS 2346.606 | RMSE 48.44 | MAE 45.79 | R2 -5.74 | CORR 0.76
VALID :: LOSS 2888.699 | RMSE 53.77 | MAE 50.79 | R2 -6.79 | CORR 0.63
Epoch 5 / 100, BEST MAE 47.728
[train] 243.7 sec [valid] 9.9 sec 
TRAI

[train] 250.7 sec [valid] 9.8 sec 
TRAIN :: LOSS 36.328 | RMSE 6.03 | MAE 4.61 | R2 0.90 | CORR 0.95
VALID :: LOSS 50.892 | RMSE 7.15 | MAE 5.54 | R2 0.86 | CORR 0.93
Epoch 41 / 100, BEST MAE 5.436
[train] 245.0 sec [valid] 10.5 sec 
TRAIN :: LOSS 34.448 | RMSE 5.87 | MAE 4.54 | R2 0.90 | CORR 0.95
VALID :: LOSS 58.649 | RMSE 7.66 | MAE 6.21 | R2 0.84 | CORR 0.93
Epoch 42 / 100, BEST MAE 5.436
[train] 253.8 sec [valid] 10.0 sec 
TRAIN :: LOSS 35.848 | RMSE 5.99 | MAE 4.66 | R2 0.90 | CORR 0.95
VALID :: LOSS 59.022 | RMSE 7.69 | MAE 6.24 | R2 0.84 | CORR 0.94
Epoch 43 / 100, BEST MAE 5.436
[train] 237.3 sec [valid] 9.9 sec 
TRAIN :: LOSS 33.509 | RMSE 5.79 | MAE 4.46 | R2 0.90 | CORR 0.95
VALID :: LOSS 46.393 | RMSE 6.82 | MAE 5.39 | R2 0.87 | CORR 0.94
Saving ...
Epoch 44 / 100, BEST MAE 5.392
[train] 236.5 sec [valid] 10.0 sec 
TRAIN :: LOSS 33.013 | RMSE 5.75 | MAE 4.48 | R2 0.91 | CORR 0.95
VALID :: LOSS 50.439 | RMSE 7.11 | MAE 5.65 | R2 0.86 | CORR 0.93
Epoch 45 / 100, BEST MAE 5.

--- Logging error ---
Traceback (most recent call last):
  File "<ipython-input-3-a6ec39c2de33>", line 27, in run_sweep
    run(cfg)
  File "G:\My Drive\brain_data\workspace\3DCNN_sMRI\sage\training\runner.py", line 68, in run
  File "G:\My Drive\brain_data\workspace\3DCNN_sMRI\sage\training\runner.py", line 19, in wrapper_fn
  File "G:\My Drive\brain_data\workspace\3DCNN_sMRI\sage\training\runner.py", line 143, in train
  File "C:\Users\pha\anaconda3\envs\cnn\lib\site-packages\torch\utils\data\dataloader.py", line 435, in __next__
    data = self._next_data()
  File "C:\Users\pha\anaconda3\envs\cnn\lib\site-packages\torch\utils\data\dataloader.py", line 475, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "C:\Users\pha\anaconda3\envs\cnn\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\pha\anaconda3\envs\cnn\lib\site-packages\torch\utils

# UNLEARNING

In [1]:
import os
import wandb

from sage.config import *
from sage.training import unlearner

from utils.misc import seed_everything, get_today

In [2]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = ['Oasis1', 'Oasis3']

cfg.unlearn = True
cfg.unlearn_cfg.encoder.name = 'resnet'
cfg.loss = 'rmse'

cfg.unlearn_cfg.opt_conf.point = 0
cfg.unlearn_cfg.domainer.num_dbs = 4 - len(cfg.unused_src)

In [3]:
name = 'Grad_zero_test_venvcnn_amp_correct'
cfg.RESULT_PATH = os.path.join(cfg.RESULT_PATH, name + get_today())

In [4]:
tags = ['amp', f'num_db-{cfg.unlearn_cfg.domainer.num_dbs}']

In [5]:
wandb.login()
wandb.init(project='3d_smri_unlearning',
           config=vars(cfg),
           name=name,
           tags=tags
  )

wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


In [6]:
unlearner.run(cfg)

Use cuda:0 as a device.
Output from encoder is 128.
TOTAL TRAIN 468 | VALID 117
Epoch 1 / 100, BEST MAE inf
 opt_age: True  opt_dom: True  opt_conf: True 


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [128]] is at version 3; expected version 2 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

## TRIALS

In [1]:
from itertools import chain

import torch
import torch.nn as nn

In [2]:
class FeatureExtractor(nn.Module):
    
    def __init__(self, cfg=None, init=False):
        
        super().__init__()
        self.cfg = cfg
        self.layers = nn.Sequential(
            nn.Linear(4, 3, bias=False),            
            nn.Linear(3, 2, bias=False),
        )
        if init:
            nn.init.constant_(self.layers[0].weight, 1)
            nn.init.constant_(self.layers[1].weight, 2)
        
        
    def forward(self, x):
        
        out = self.layers(x)
        return out

In [3]:
class Predictor(nn.Module):
    
    def __init__(self, cfg=None, last_node=1, init=False):
        
        super().__init__()
        self.cfg = cfg
        self.layers = nn.Sequential(
            nn.Linear(2, last_node, bias=False),
        )
        if init:  
            nn.init.constant_(self.layers[0].weight, 3)

    def forward(self, x):
        
        out = self.layers(x)
        return out

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fe = FeatureExtractor(init=True).to(device)
reg = Predictor(last_node=1, init=True).to(device)
clf = Predictor(last_node=2, init=True).to(device)

In [21]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler

n_samples = 4
n_features = 4
n_classes = 2
X, y = make_regression(
    n_samples=n_samples,
    n_features=n_features,
)
y = StandardScaler().fit_transform(y.reshape(-1, 1)).reshape(-1)

sample_batch = torch.tensor(X, dtype=torch.float).to(device)
sample_float = torch.tensor(y, dtype=torch.float).to(device)

In [44]:
# sample_batch = torch.rand((256, 4)).to(device)
# # sample_batch = torch.ones((256, 4)).to(device)
# sample_float = torch.rand((256,)).to(device)
sample_label = torch.randint(low=0, high=2, size=(4,)).to(device)
sample_label_onehot = torch.zeros((n_samples, n_classes)).to(device)
sample_label_onehot[range(n_samples), sample_label] = 1

In [45]:
def get_optimizer(models):

    if isinstance(models, nn.Module):
        params = models.parameters()
    else:
        params = list(chain(*([list(m.parameters()) for m in models])))

    optimizer = torch.optim.Adam(params, lr=1e-4)
    optimizer.zero_grad()
    
    return optimizer

class confusion_loss(nn.Module):
    def __init__(self, task=0):
        super(confusion_loss, self).__init__()
        self.task = task

    def forward(self, x, target):
        # We only care about x
        log = torch.log(x)
        log_sum = torch.sum(log, dim=1)
        normalised_log_sum = torch.div(log_sum,  x.size()[1])
        loss = torch.mul(torch.sum(normalised_log_sum, dim=0), -1)
        return loss

In [46]:
opt_reg = get_optimizer([fe, reg])
opt_dom = get_optimizer([clf])
opt_conf = get_optimizer([fe])

loss_fn_reg = nn.MSELoss()
loss_fn_dom = nn.BCEWithLogitsLoss()
loss_fn_conf = confusion_loss()

In [47]:
losses = list()
for e in range(100):
    
    # TRAIN
    fe.train()
    reg.train()
    clf.train()
    
    # 1. AGE PREDICTION
    opt_reg.zero_grad()
    embed = fe(sample_batch)
    y_pred = reg(embed)
    loss = loss_fn_reg(y_pred.squeeze(), sample_float)
    losses.append(loss.item())
    
    loss.backward()
    opt_reg.step()
    
    # 2. DOMAIN PREDICTION
    opt_dom.zero_grad()
    d_pred = clf(embed.detach())
    loss_dom = loss_fn_dom(d_pred, sample_label_onehot)
    
    loss_dom.backward()
    opt_dom.step()
    
    # 3. UNLEARNING
    opt_conf.zero_grad()
    d_conf = clf(embed.detach())
    loss_conf = loss_fn_conf(d_conf, sample_label)
    
    loss_conf.backward()
    opt_conf.step()

In [48]:
d_pred

tensor([[ -5.6110,  -5.6116],
        [ -8.2645,  -8.2653],
        [-30.3303, -30.3333],
        [-36.0828, -36.0864]], device='cuda:0', grad_fn=<MmBackward>)

In [49]:
list(reg.parameters())[0].grad

tensor([[189.7018, 189.7018]], device='cuda:0')

In [50]:
fe.state_dict()

OrderedDict([('layers.0.weight',
              tensor([[1.0075, 0.9903, 0.9897, 1.0097],
                      [1.0075, 0.9903, 0.9897, 1.0097],
                      [1.0075, 0.9903, 0.9897, 1.0097]], device='cuda:0')),
             ('layers.1.weight',
              tensor([[1.9898, 1.9898, 1.9898],
                      [1.9898, 1.9898, 1.9898]], device='cuda:0'))])

In [52]:
vars(opt_reg)

{'defaults': {'lr': 0.0001,
  'betas': (0.9, 0.999),
  'eps': 1e-08,
  'weight_decay': 0,
  'amsgrad': False},
 'state': defaultdict(dict,
             {Parameter containing:
              tensor([[1.0075, 0.9903, 0.9897, 1.0097],
                      [1.0075, 0.9903, 0.9897, 1.0097],
                      [1.0075, 0.9903, 0.9897, 1.0097]], device='cuda:0', requires_grad=True): {'step': 100,
               'exp_avg': tensor([[  -3.8777,   28.1076,  629.8959, -265.3368],
                       [  -3.8777,   28.1076,  629.8959, -265.3368],
                       [  -3.8777,   28.1076,  629.8959, -265.3368]], device='cuda:0'),
               'exp_avg_sq': tensor([[7.2148e+00, 7.3972e+01, 3.9607e+04, 6.9057e+03],
                       [7.2148e+00, 7.3972e+01, 3.9607e+04, 6.9057e+03],
                       [7.2148e+00, 7.3972e+01, 3.9607e+04, 6.9057e+03]], device='cuda:0')},
              Parameter containing:
              tensor([[1.9898, 1.9898, 1.9898],
                      [1.9898,

In [62]:
next(reg.parameters()).grad

tensor([[189.7018, 189.7018]], device='cuda:0')

In [56]:
opt_reg.state

defaultdict(dict,
            {Parameter containing:
             tensor([[1.0075, 0.9903, 0.9897, 1.0097],
                     [1.0075, 0.9903, 0.9897, 1.0097],
                     [1.0075, 0.9903, 0.9897, 1.0097]], device='cuda:0', requires_grad=True): {'step': 100,
              'exp_avg': tensor([[  -3.8777,   28.1076,  629.8959, -265.3368],
                      [  -3.8777,   28.1076,  629.8959, -265.3368],
                      [  -3.8777,   28.1076,  629.8959, -265.3368]], device='cuda:0'),
              'exp_avg_sq': tensor([[7.2148e+00, 7.3972e+01, 3.9607e+04, 6.9057e+03],
                      [7.2148e+00, 7.3972e+01, 3.9607e+04, 6.9057e+03],
                      [7.2148e+00, 7.3972e+01, 3.9607e+04, 6.9057e+03]], device='cuda:0')},
             Parameter containing:
             tensor([[1.9898, 1.9898, 1.9898],
                     [1.9898, 1.9898, 1.9898]], device='cuda:0', requires_grad=True): {'step': 100,
              'exp_avg': tensor([[95.5300, 95.5300, 95.5300],
 

In [49]:
from graphviz import Digraph
from torch.autograd import Variable
import torch


def make_dot(var, params=None):
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '(' + (', ').join(['%d' % v for v in size]) + ')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
                dot.edge(str(id(var.grad_fn)), str(id(var)))
                var = var.grad_fn
            if hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var)
    return dot

In [57]:
fe.state_dict()

OrderedDict([('layers.0.weight',
              tensor([[1., 1., 1., 1.],
                      [1., 1., 1., 1.],
                      [1., 1., 1., 1.]], device='cuda:0')),
             ('layers.3.weight',
              tensor([[2., 2., 2.],
                      [2., 2., 2.]], device='cuda:0'))])

In [56]:
g = make_dot(loss, params=fe.state_dict())
g.view()

TypeError: 'odict_values' object is not subscriptable

## BACKWARD

In [30]:
a = t(2., requires_grad=True)
b = a ** 2
c = 2 * b
d = 4 * b

In [31]:
c.backward(retain_graph=True)
print(a.grad)

tensor(8.)


In [32]:
d.backward()
a.grad

tensor(24.)

In [33]:
a = t(2., requires_grad=True)
b = a ** 2
d = 4 * b
d.backward()
print(a.grad)

tensor(16.)
